<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#SpecD" data-toc-modified-id="SpecD-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>SpecD</a></span><ul class="toc-item"><li><span><a href="#Aggregating" data-toc-modified-id="Aggregating-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Aggregating</a></span></li><li><span><a href="#Filter-by-sparsity" data-toc-modified-id="Filter-by-sparsity-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Filter by sparsity</a></span></li><li><span><a href="#Zero-estimation" data-toc-modified-id="Zero-estimation-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Zero estimation</a></span></li><li><span><a href="#clr-transformation" data-toc-modified-id="clr-transformation-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>clr transformation</a></span></li><li><span><a href="#New-phyloseq-obj." data-toc-modified-id="New-phyloseq-obj.-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>New phyloseq obj.</a></span></li></ul></li><li><span><a href="#IndD" data-toc-modified-id="IndD-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>IndD</a></span><ul class="toc-item"><li><span><a href="#Aggregating" data-toc-modified-id="Aggregating-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Aggregating</a></span></li><li><span><a href="#Filter-by-sparsity" data-toc-modified-id="Filter-by-sparsity-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Filter by sparsity</a></span></li><li><span><a href="#Zero-estimation" data-toc-modified-id="Zero-estimation-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Zero estimation</a></span></li><li><span><a href="#clr-transformation" data-toc-modified-id="clr-transformation-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>clr transformation</a></span></li><li><span><a href="#New-phyloseq-obj" data-toc-modified-id="New-phyloseq-obj-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>New phyloseq obj</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Create coda-transformed dataset
  * accounting for compositional data
* Method
  * impute zeros with zCompositions
  * perform clr transformation on counts
    * `compositions::clr`

# Var

In [1]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/'

# microbiome
## 16S-arch phyloseq object (one sample per species)
physeq_arch_SpecD_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_SpecD.RDS'
## 16S-arch phyloseq object (one sample per individual)
physeq_arch_IndD_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_IndD.RDS'

# params
threads = 8
my_seed = 8371


# Init

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)

library(phyloseq)
library(zCompositions)

library(LeyLabRMisc)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: NADA

Loading required package: survival


Attaching package: ‘NADA’


The following object is masked from ‘package:stats’:

    cor


Loading required package: truncnorm



In [3]:
df.dims()
set.seed(my_seed)
make_dir(work_dir)

Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/ 


# Load

In [4]:
# phyloseq
physeq_arch_SpecD = readRDS(physeq_arch_SpecD_file)
physeq_arch_SpecD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1151 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 1151 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1151 tips and 1150 internal nodes ]

In [5]:
# phyloseq
physeq_arch_IndD = readRDS(physeq_arch_IndD_file)
physeq_arch_IndD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1891 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 1891 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1891 tips and 1888 internal nodes ]

# SpecD

## Aggregating

In [6]:
physeq_arch_SpecD = tax_glom(physeq_arch_SpecD, taxrank='Genus')
physeq_arch_SpecD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 82 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 82 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 82 tips and 81 internal nodes ]

## Filter by sparsity

In [7]:
# filter by sparsity
physeq_arch_SpecD = physeq_arch_SpecD %>%
    filter_taxa(function(x) sum(x > 0) >= 2, TRUE) %>%
    prune_samples(sample_sums(.) > 0, .)
physeq_arch_SpecD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 42 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 42 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 42 tips and 41 internal nodes ]

In [8]:
# otu table
otu = physeq_arch_SpecD %>%
    phyloseq2df(otu_table) %>%
    as.data.frame
rownames(otu) = otu$OTU
otu$OTU = NULL
otu = otu %>% t %>% as.data.frame
otu

,c3b19986023201183d07b2bf6dc9eaab,e6ddd5efa2e9229aa04f33e832450fa8,5887dc080d027d30580310a1eb1f3166,66cc60d5322fdeaf48cc8bdd82effea2,6668cc03267d4f04b5a34d676c1db57a,252850216e7d0123bfafd7eed609799d,dc2d500b34b0dc2adb147b190d3b337a,19fff4ad5946206827aa5d91e47fbcf1,cfd2bb81e97edddddf1a843a0d6999fa,51b922408976c428c8b582162eaefb1c,⋯,2987f396047df01622b365b3b667a4eb,fef8ef65bd3598e9f1fec3bd32a9b4bd,9e6f5751d7458b48c1fd8272e3683ded,2aae9c46a33f7677efc6534f91a980bc,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a,b7ed842d3fbe5d8d33504a0a765409ae,e94f4096695b98afaec1d5a4517b0c87,250a31c11dbecf1e2ef705424fdaa7a7,188795b8eb1edbd3334be2676ca05391
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0,0,2,0,0,0,0,0,0,507,⋯,0,0,6,0,0,14,0,0,2599,0
F44_Fallow_Deer,0,0,0,0,0,0,0,0,0,0,⋯,127,0,443,0,0,5074,0,693,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X405_Nile_Crocodile,0,3,0,0,0,10,0,0,0,0,⋯,0,0,0,0,0,0,0,0,25,5
X67_European_Roe,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,709,0,12,0,0


## Zero estimation

In [9]:
# sparsity (%)
otu %>%
    apply(2, function(x) sum(x == 0) / length(x) * 100) %>%
    summary_x('sparsity')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
sparsity,3.636364,87.27273,95.90909,87.1645,97.95455,98.18182,36.947,15.084


In [10]:
# inferring zeros
## warning: a bit slow
otu = cmultRepl(otu, output = 'p-counts')
otu

No. corrected values:  1079 


,c3b19986023201183d07b2bf6dc9eaab,e6ddd5efa2e9229aa04f33e832450fa8,5887dc080d027d30580310a1eb1f3166,66cc60d5322fdeaf48cc8bdd82effea2,6668cc03267d4f04b5a34d676c1db57a,252850216e7d0123bfafd7eed609799d,dc2d500b34b0dc2adb147b190d3b337a,19fff4ad5946206827aa5d91e47fbcf1,cfd2bb81e97edddddf1a843a0d6999fa,51b922408976c428c8b582162eaefb1c,⋯,2987f396047df01622b365b3b667a4eb,fef8ef65bd3598e9f1fec3bd32a9b4bd,9e6f5751d7458b48c1fd8272e3683ded,2aae9c46a33f7677efc6534f91a980bc,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a,b7ed842d3fbe5d8d33504a0a765409ae,e94f4096695b98afaec1d5a4517b0c87,250a31c11dbecf1e2ef705424fdaa7a7,188795b8eb1edbd3334be2676ca05391
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0.03419676,0.06313249,2.0000000,0.2183332,0.3735339,0.04997988,3.961563,0.2946183,0.2262247,507.000000,⋯,1.960887,0.1400200,6,1.652707,1.11008,14,0.7000999,3.239116,2599.000000,1.652707
F44_Fallow_Deer,0.03389148,0.06256889,0.1173167,0.2163841,0.3701992,0.04953370,3.926198,0.2919881,0.2242052,3.031984,⋯,127.000000,0.2572004,443,2.849491,1.10017,5074,1.2860019,693.000000,9.476579,3.146694
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X405_Nile_Crocodile,0.01668630,3.0000000,0.05776027,0.1065356,0.1822657,10.00000000,1.933044,0.1437589,0.1103863,0.4854015,⋯,0.2747381,0.01961807,0.5509096,0.2315592,0.3088962,1.285456,0.09809037,0.4538296,25.000000,5.0000000
X67_European_Roe,0.01992228,0.0367796,0.06896175,0.1271961,0.2176126,0.02911718,2.307920,0.1716381,0.1317936,0.7344780,⋯,0.4157158,0.02968479,0.8336005,0.3503803,0.6467079,709.000000,0.14842395,12.0000000,2.095918,0.3503803


In [11]:
# check
## sparsity (%)
otu %>%
    apply(1, function(x) sum(x == 0) / length(x) * 100) %>%
    summary_x('sparsity')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
sparsity,0,0,0,0,0,0,0,0


## clr transformation

In [12]:
# transforming
otu = compositions::clr(otu) %>%
    as.data.frame
otu

,c3b19986023201183d07b2bf6dc9eaab,e6ddd5efa2e9229aa04f33e832450fa8,5887dc080d027d30580310a1eb1f3166,66cc60d5322fdeaf48cc8bdd82effea2,6668cc03267d4f04b5a34d676c1db57a,252850216e7d0123bfafd7eed609799d,dc2d500b34b0dc2adb147b190d3b337a,19fff4ad5946206827aa5d91e47fbcf1,cfd2bb81e97edddddf1a843a0d6999fa,51b922408976c428c8b582162eaefb1c,⋯,2987f396047df01622b365b3b667a4eb,fef8ef65bd3598e9f1fec3bd32a9b4bd,9e6f5751d7458b48c1fd8272e3683ded,2aae9c46a33f7677efc6534f91a980bc,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a,b7ed842d3fbe5d8d33504a0a765409ae,e94f4096695b98afaec1d5a4517b0c87,250a31c11dbecf1e2ef705424fdaa7a7,188795b8eb1edbd3334be2676ca05391
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,-3.787002,-3.173898,0.2817694,-1.933111,-1.3961243,-3.407512,0.965261,-1.633453,-1.897604,5.817133,⋯,0.2620191,-2.377348,1.380382,0.09103657,-0.3069461,2.227680,-0.7679100,0.7639226,7.451504,0.09103657
F44_Fallow_Deer,-3.330035,-2.716931,-2.0883219,-1.476144,-0.9391573,-2.950545,1.422228,-1.176486,-1.440637,1.163774,⋯,4.8987437,-1.303343,6.148126,1.10169710,0.1500209,8.586441,0.3060947,6.5955866,2.303380,1.20090883
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X405_Nile_Crocodile,-3.778480,1.413299,-2.536767,-1.924589,-1.3876026,2.617272,0.9737828,-1.6249308,-1.8890824,-0.4080918,⋯,-0.9772499,-3.616617,-0.2814975,-1.1482325,-0.8600630,0.5658004,-2.0071791,-0.4753465,3.533563,1.9241249
X67_European_Roe,-2.569378,-1.956274,-1.327665,-0.715487,-0.1785006,-2.189889,2.1828848,-0.4158288,-0.6799804,1.0379429,⋯,0.4687847,-2.170582,1.1645372,0.2978021,0.9106777,7.9103937,-0.5611444,3.8314448,2.086530,0.2978021


In [13]:
# summary
otu %>% rowSums %>% summary
otu %>% colSums %>% summary

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-2.598e-14 -3.740e-15  3.608e-16 -1.711e-16  3.969e-15  2.065e-14 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-333.20 -166.52   20.96    0.00  132.43  652.24 

## New phyloseq obj.

In [14]:
# new phyloseq object
physeq_arch_SpecD_clr = phyloseq(otu_table(otu %>% t %>% as.data.frame, 
                                           taxa_are_rows=TRUE),
                                tax_table(physeq_arch_SpecD),
                                sample_data(physeq_arch_SpecD),
                                phy_tree(physeq_arch_SpecD))
physeq_arch_SpecD_clr

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 42 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 42 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 42 tips and 41 internal nodes ]

In [15]:
# writing new file
physeq_arch_SpecD_clr_file = file.path(dirname(physeq_arch_SpecD_file),
                                      'physeq_arch_SpecD_clr-genus.RDS')
saveRDS(physeq_arch_SpecD_clr, physeq_arch_SpecD_clr_file)
cat('File written:', physeq_arch_SpecD_clr_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_SpecD_clr-genus.RDS 


# IndD

## Aggregating

In [16]:
physeq_arch_IndD = tax_glom(physeq_arch_IndD, taxrank='Genus')
physeq_arch_IndD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 90 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 90 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 90 tips and 89 internal nodes ]

## Filter by sparsity

In [17]:
# filter by sparsity
physeq_arch_IndD = physeq_arch_IndD %>%
    filter_taxa(function(x) sum(x > 0) >= 2, TRUE) %>%
    prune_samples(sample_sums(.) > 0, .)
physeq_arch_IndD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 57 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 57 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 57 tips and 56 internal nodes ]

In [18]:
# otu table
otu = physeq_arch_IndD %>%
    phyloseq2df(otu_table) %>%
    as.data.frame
rownames(otu) = otu$OTU
otu$OTU = NULL
otu = otu %>% t %>% as.data.frame
otu

,c3b19986023201183d07b2bf6dc9eaab,e6ddd5efa2e9229aa04f33e832450fa8,5887dc080d027d30580310a1eb1f3166,c0eb38cd52d8be27c54de8a8b06f9a9a,bfb96cf07d2fec50b18d3a9665ba6e80,66cc60d5322fdeaf48cc8bdd82effea2,f906aca9a9c84f01cbd8a874cce7e6fe,1936b1a270b058c5754be0addc7c7c31,252850216e7d0123bfafd7eed609799d,7950a70a07006f7c7b0c6f02c46a93af,⋯,092547e9f87986180c27587156336bcb,2987f396047df01622b365b3b667a4eb,fef8ef65bd3598e9f1fec3bd32a9b4bd,b9abd819fdd733c57642461d1c2247a6,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a,b7ed842d3fbe5d8d33504a0a765409ae,1f1f87cab902b66753b8bbd81e69a7d4,250a31c11dbecf1e2ef705424fdaa7a7,188795b8eb1edbd3334be2676ca05391
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0,0,2,0,0,0,0,0,0,0,⋯,6,0,0,0,0,14,0,0,2599,0
F35_Red_Deer,0,0,0,2,0,0,0,0,0,0,⋯,81,0,0,0,0,930,0,305,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X431_Quagga,0,0,0,0,0,0,0,0,0,0,⋯,129,0,0,138,0,30,0,0,0,0
X67_European_Roe,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,709,0,12,0,0


## Zero estimation

In [19]:
# sparsity (%)
otu %>%
    apply(1, function(x) sum(x == 0) / length(x) * 100) %>%
    summary_x('sparsity')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
sparsity,71.92982,89.47368,91.22807,90.38407,94.73684,98.24561,9.124,3.725


In [20]:
# inferring zeros
## warning: a bit slow
otu = cmultRepl(otu, output = 'p-counts')
otu

No. corrected values:  2254 


,c3b19986023201183d07b2bf6dc9eaab,e6ddd5efa2e9229aa04f33e832450fa8,5887dc080d027d30580310a1eb1f3166,c0eb38cd52d8be27c54de8a8b06f9a9a,bfb96cf07d2fec50b18d3a9665ba6e80,66cc60d5322fdeaf48cc8bdd82effea2,f906aca9a9c84f01cbd8a874cce7e6fe,1936b1a270b058c5754be0addc7c7c31,252850216e7d0123bfafd7eed609799d,7950a70a07006f7c7b0c6f02c46a93af,⋯,092547e9f87986180c27587156336bcb,2987f396047df01622b365b3b667a4eb,fef8ef65bd3598e9f1fec3bd32a9b4bd,b9abd819fdd733c57642461d1c2247a6,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a,b7ed842d3fbe5d8d33504a0a765409ae,1f1f87cab902b66753b8bbd81e69a7d4,250a31c11dbecf1e2ef705424fdaa7a7,188795b8eb1edbd3334be2676ca05391
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0.03617861,0.08441676,2.0000000,0.03135479,0.03376670,0.2074240,0.1977764,1.1287726,0.04582624,1.787223,⋯,6,1.967987,0.14052698,1.6586909,1.9922354,14,0.7026349,1.489229,2599.0000000,0.7623347
F35_Red_Deer,0.02803368,0.06541191,0.1140036,2.00000000,0.02616476,0.1607264,0.1532508,0.8746507,0.03550932,1.384864,⋯,81,1.072973,0.07661717,0.9043402,0.9868153,930,0.3830858,305.000000,0.3044887,0.4156350
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X431_Quagga,0.03099892,0.07233082,0.12606229,0.02686573,0.02893233,0.17772716,0.16946078,0.9671664,0.03926530,1.5313468,⋯,129.0000000,1.3912466,0.09934398,138.0000000,1.7070074,30,0.4967199,1.052794,0.3948086,0.5389240
X67_European_Roe,0.01671186,0.03899434,0.06796157,0.01448361,0.01559774,0.09581467,0.09135817,0.5214101,0.02116836,0.8255659,⋯,0.5819093,0.4153611,0.02965946,0.3500814,0.3820085,709,0.1482973,12.000000,0.1178714,0.1608975


## clr transformation

In [21]:
# transforming
otu = compositions::clr(t(otu)) %>%
    t %>% as.data.frame
otu

,c3b19986023201183d07b2bf6dc9eaab,e6ddd5efa2e9229aa04f33e832450fa8,5887dc080d027d30580310a1eb1f3166,c0eb38cd52d8be27c54de8a8b06f9a9a,bfb96cf07d2fec50b18d3a9665ba6e80,66cc60d5322fdeaf48cc8bdd82effea2,f906aca9a9c84f01cbd8a874cce7e6fe,1936b1a270b058c5754be0addc7c7c31,252850216e7d0123bfafd7eed609799d,7950a70a07006f7c7b0c6f02c46a93af,⋯,092547e9f87986180c27587156336bcb,2987f396047df01622b365b3b667a4eb,fef8ef65bd3598e9f1fec3bd32a9b4bd,b9abd819fdd733c57642461d1c2247a6,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a,b7ed842d3fbe5d8d33504a0a765409ae,1f1f87cab902b66753b8bbd81e69a7d4,250a31c11dbecf1e2ef705424fdaa7a7,188795b8eb1edbd3334be2676ca05391
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0.3533185,0.32246438,2.91428908,0.3254381,0.3703154,0.3612832,0.30778200,0.27413385,0.3629528,0.29043538,⋯,0.1665158,-0.03874821,0.4547095,0.2546676,0.73849765,-2.920287,-3.236448,-2.117767,8.7026399,0.4149247
F35_Red_Deer,0.0982570,0.06740287,0.04961681,4.4809734,0.1152539,0.1062217,0.05272048,0.01907233,0.1078913,0.03537386,⋯,2.7692054,-0.64532655,-0.1518688,-0.3519108,0.03596791,1.275840,-3.843026,3.204286,-0.3493636,-0.1916536
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X431_Quagga,0.1988029,0.1679488,0.1501628,0.1709225,0.2157999,0.2067676,0.1532664,0.1196183,0.2084372,0.1359198,⋯,3.234569,-0.3855593,0.1078984,4.675893,0.5839821,-2.158147,-3.583259,-2.46457812,-0.08959639,0.0681136
X67_European_Roe,-0.4190308,-0.4498850,-0.4676710,-0.4469112,-0.4020339,-0.4110661,-0.4645673,-0.4982155,-0.4093965,-0.4819140,⋯,-2.166684,-1.5943665,-1.1009088,-1.300951,-0.9130721,1.004511,-4.792066,-0.03111916,-1.29840360,-1.1406936


In [22]:
# summary
otu %>% rowSums %>% summary
otu %>% colSums %>% summary

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-92.235 -26.878   8.046   0.000  27.231  58.193 

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-2.611e-13 -5.373e-14 -2.581e-15 -1.383e-14  3.109e-14  2.420e-13 

## New phyloseq obj

In [23]:
# new phyloseq object
physeq_arch_IndD_clr = phyloseq(otu_table(otu %>% t %>% as.data.frame, 
                                          taxa_are_rows=TRUE),
                                tax_table(physeq_arch_IndD),
                                sample_data(physeq_arch_IndD),
                                phy_tree(physeq_arch_IndD))
physeq_arch_IndD_clr

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 57 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 57 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 57 tips and 56 internal nodes ]

In [24]:
# writing new file
physeq_arch_IndD_clr_file = file.path(dirname(physeq_arch_IndD_file),
                                      'physeq_arch_IndD_clr-genus.RDS')
saveRDS(physeq_arch_IndD_clr, physeq_arch_IndD_clr_file)
cat('File written:', physeq_arch_IndD_clr_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_IndD_clr-genus.RDS 


# sessionInfo

In [25]:
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.4 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/phyloseq-eco/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] LeyLabRMisc_0.1.6   zCompositions_1.3.4 truncnorm_1.0-8    
 [4] NADA_1.6-1.1        survival_3.1-8      MASS_7.3-51.5      
 [7] phyloseq_1.30.0     ggplot2_3.2.1       tidyr_1.0.0        
[10] dplyr_0.8.5        

loaded via a namespace (and not attached):
 [1] Biobase_2.